In [2]:
import os
import sys
from six.moves import urllib
import tarfile
import tensorflow as tf
import numpy as np


def maybe_download_and_extract(dest_directory, url):
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    file_name = 'cifar-100-binary.tar.gz'
    file_path = os.path.join(dest_directory, file_name)
    # if have not downloaded yet
    if not os.path.exists(file_path):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\r%.1f%%' % 
                (float(count * block_size) / float(total_size) * 100.0))
            sys.stdout.flush()  # flush the buffer

    print('>> Downloading %s ...' % file_name)
    file_path, _ = urllib.request.urlretrieve(url, file_path, _progress)
    file_size = os.stat(file_path).st_size
    print('\r>> Total %d bytes' % file_size)
    extracted_dir_path = os.path.join(dest_directory, 'cifar-10-batches-bin')
    if not os.path.exists(extracted_dir_path):
    # Open for reading with gzip compression, then extract all
        tarfile.open(file_path, 'r:gz').extractall(dest_directory)
    print('>> Done')

# <1 x label> <3072 x pixel>
DATA_URL = 'http://www.cs.toronto.edu/~kriz/cifar-100-binary.tar.gz'
DEST_DIRECTORY = 'dataset/cifar10'
DATA_DIRECTORY = DEST_DIRECTORY + '/cifar-10-batches-bin'

IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32
IMAGE_DEPTH = 3

# 圖片尺寸 24
IMAGE_SIZE_CROPPED = 24
BATCH_SIZE = 128

# 10個類別
NUM_CLASSES = 10

# Label 只有一個字節
LABEL_BYTES = 1

# (IMAGE_HEIGHT) * (IMAGE_WIDTH) * (IMAGE_DEPTH)
IMAGE_BYTES = 32 * 32 * 3

# 樣本數目
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

# download it
# maybe_download_and_extract(DEST_DIRECTORY, DATA_URL)


In [2]:
from tensorflow.contrib.data import FixedLengthRecordDataset, Iterator

def cifar10_record_distort_parser(record):
    ''' Parse the record into label, cropped and distorted image
    -----
    Args:
        record: 
            a record containing label and image.
    Returns:
        label: 
            the label in the record.
        image: 
            the cropped and distorted image in the record.
    '''
    # TODO1
    
    # 解碼器 
    record_uint8 = tf.decode_raw(record, tf.uint8)
    
    label = tf.cast(tf.strided_slice(record_uint8, [0], [LABEL_BYTES]), tf.int32)
    
    depth_major = tf.reshape(
      tf.strided_slice(record_uint8, [LABEL_BYTES],
                       [LABEL_BYTES + IMAGE_BYTES]),
      [IMAGE_DEPTH, IMAGE_HEIGHT, IMAGE_WIDTH])
    
    image = tf.transpose(depth_major, [1, 2, 0])
    
    float_image = tf.cast(image, tf.float32)
    
    height = IMAGE_SIZE_CROPPED
    width = IMAGE_SIZE_CROPPED
    
    distorted_image = tf.random_crop(float_image, [height, width, 3])
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
    distorted_image = tf.image.random_contrast(
      distorted_image, lower=0.2, upper=1.8)
    # standardization: subtract off the mean and divide by the variance of the pixels
    distorted_image = tf.image.per_image_standardization(distorted_image)
    # Set the shapes of tensors.
    distorted_image.set_shape([height, width, 3])
    label.set_shape([1])
    return label, distorted_image


def cifar10_record_crop_parser(record):
    ''' Parse the record into label, cropped image
    -----
    Args:
        record: 
            a record containing label and image.
    Returns:
        label: 
            the label in the record.
        image: 
            the cropped image in the record.
    '''
    # TODO2
    record_uint8 = tf.decode_raw(record, tf.uint8)
    
    label = tf.cast(tf.strided_slice(record_uint8, [0], [LABEL_BYTES]), tf.int32)
    
    depth_major = tf.reshape(
      tf.strided_slice(record_uint8, [LABEL_BYTES],
                       [LABEL_BYTES + IMAGE_BYTES]),
      [IMAGE_DEPTH, IMAGE_HEIGHT, IMAGE_WIDTH])
    
    image = tf.transpose(depth_major, [1, 2, 0])
    
    float_image = tf.cast(image, tf.float32)
    
    height = IMAGE_SIZE_CROPPED
    width = IMAGE_SIZE_CROPPED
    
    distorted_image = tf.random_crop(float_image, [height, width, 3])
    distorted_image.set_shape([height, width, 3])
    label.set_shape([1])
    return label, distorted_image


def cifar10_iterator(filenames, batch_size, cifar10_record_parser):
    ''' Create a dataset and return a tf.contrib.data.Iterator 
    which provides a way to extract elements from this dataset.
    -----
    Args:
        filenames: 
            a tensor of filenames.
        batch_size: 
            batch size.
    Returns:
        iterator: 
            an Iterator providing a way to extract elements from the created dataset.
        output_types: 
            the output types of the created dataset.
        output_shapes: 
            the output shapes of the created dataset.
    '''
    record_bytes = LABEL_BYTES + IMAGE_BYTES
    # (5) reader for cifar10 file format
    dataset = FixedLengthRecordDataset(filenames, record_bytes)
    # TODO3
    # tips: use dataset.map with cifar10_record_parser(record)
    #       output_types = dataset.output_types
    #       output_shapes = dataset.output_shapes
    
    dataset = dataset.map(cifar10_record_parser,num_parallel_calls=16)

    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(
        NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * min_fraction_of_examples_in_queue)
    
    dataset = dataset.repeat()
    batch_dataset = dataset.batch(batch_size)

    
    iterator = batch_dataset.make_one_shot_iterator()
    output_types = batch_dataset.output_types
    output_shapes = batch_dataset.output_shapes
    return iterator, output_types, output_shapes

In [3]:
model_hps_cifar = tf.contrib.training.HParams(
  image_size = IMAGE_SIZE_CROPPED,
  batch_size = BATCH_SIZE,
  num_classes = NUM_CLASSES,
  num_training_example = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN,
  num_epoch_per_decay = 350.0,
  init_lr = 0.1,
  moving_average_decay = 0.9999,
  ckpt_dir = './model/'
)

tf.reset_default_graph()
# data_batch_0.bin ~ data_batch_5
training_files = [
    os.path.join(DATA_DIRECTORY, 'data_batch_%d.bin' % i) for i in range(1, 6)]
testing_files = [os.path.join(DATA_DIRECTORY, 'test_batch.bin')]

filenames_train = tf.constant(training_files)
filenames_test = tf.constant(testing_files)

iterator_train, types, shapes = cifar10_iterator(filenames_train, BATCH_SIZE,
                                                 cifar10_record_distort_parser)


iterator_test,_,_ = cifar10_iterator(filenames_test, BATCH_SIZE,
                                    cifar10_record_crop_parser)

# use to handle training and testing
handle = tf.placeholder(tf.string, shape=[])
iterator = Iterator.from_string_handle(handle, types, shapes)
labels_images_pairs = iterator.get_next()

# CNN model
model = CNN_Model(model_hps_cifar)

with tf.device('/cpu:0'):
    labels, images = labels_images_pairs
    print(images)
    print(labels)
    labels = tf.reshape(labels, [BATCH_SIZE])
    images = tf.reshape(
      images, [BATCH_SIZE, IMAGE_SIZE_CROPPED, IMAGE_SIZE_CROPPED, IMAGE_DEPTH])
    print('Shape of cropped image:', images.shape)
    print('Shape of label:', labels.shape)
with tf.variable_scope('model'):
    logits = model.inference(images)
# train
global_step = tf.contrib.framework.get_or_create_global_step()
total_loss = model.loss(logits, labels)
train_op = model.train(total_loss, global_step)
# test
top_k_op = tf.nn.in_top_k(logits, labels, 1)


Instructions for updating:
Use `tf.data.FixedLengthRecordDataset`.
Tensor("IteratorGetNext:1", shape=(?, 24, 24, 3), dtype=float32)
Tensor("IteratorGetNext:0", shape=(?, 1), dtype=int32)
Shape of cropped image: (128, 24, 24, 3)
Shape of label: (128,)
Instructions for updating:
Please switch to tf.train.get_or_create_global_step


In [4]:
%%time

# TODO4:
# 1. train the CNN model 10 epochs
# 2. show the loss per epoch
# 3. get the accuracy of this 10-epoch model
# 4. measure the time using '%%time' instruction
# tips:
# use placeholder handle to determine if training or testing. 

# op for training
NUM_EPOCH = 10
NUM_BATCH_PER_EPOCH = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN // BATCH_SIZE
ckpt_dir = './model/'

# train
saver = tf.train.Saver()
with tf.Session() as sess:
    train_handle = sess.run(iterator_train.string_handle())
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if (ckpt and ckpt.model_checkpoint_path):
        saver.restore(sess, ckpt.model_checkpoint_path)
        # assume the name of checkpoint is like '.../model.ckpt-1000'
        gs = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
        sess.run(tf.assign(global_step, gs))
    else:
        # no checkpoint found
        sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    loss = []
    
    for i in range(NUM_EPOCH):
        _loss = []
        for _ in range(NUM_BATCH_PER_EPOCH):
            l, _ = sess.run([total_loss, train_op],feed_dict={handle: train_handle})
            _loss.append(l)
        loss_this_epoch = np.sum(_loss)
        gs = global_step.eval()
        print('loss of epoch %d: %f' % (gs / NUM_BATCH_PER_EPOCH, loss_this_epoch))
        loss.append(loss_this_epoch)
        saver.save(sess, ckpt_dir + 'model.ckpt', global_step=gs)
    coord.request_stop()
    coord.join(threads)
  
    print('Done')
    

loss of epoch 1: 1519.412231
loss of epoch 2: 1187.492188
loss of epoch 3: 967.885376
loss of epoch 4: 810.788757
loss of epoch 5: 698.386597
loss of epoch 6: 617.807861
loss of epoch 7: 553.512634
loss of epoch 8: 509.955994
loss of epoch 9: 473.897217
loss of epoch 10: 452.237976
Done
CPU times: user 2h 1min 51s, sys: 12min 10s, total: 2h 14min 2s
Wall time: 35min 22s


In [5]:
%%time
variables_to_restore = model.ema.variables_to_restore()
saver = tf.train.Saver(variables_to_restore)


with tf.Session() as sess:
    test_handle = sess.run(iterator_test.string_handle())
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    saver.restore(sess, ckpt.model_checkpoint_path)
    # assume the name of checkpoint is like '.../model.ckpt-1000'
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    num_iter = NUM_EXAMPLES_PER_EPOCH_FOR_EVAL // BATCH_SIZE
    total_sample_count = num_iter * BATCH_SIZE
    true_count = 0
    for _ in range(num_iter):
        predictions = sess.run(top_k_op,feed_dict={handle: test_handle})
        true_count += np.sum(predictions)
    print('Accurarcy: %d/%d = %f' % (true_count, total_sample_count,
                                     true_count / total_sample_count))
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Restoring parameters from ./model/model.ckpt-3900
Accurarcy: 4447/9984 = 0.445413
CPU times: user 46.9 s, sys: 2.28 s, total: 49.2 s
Wall time: 8.87 s
